In [1]:
%run _standard_imports.ipynb
import pandas as pd

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.3
scipy 0.18.1
pandas 0.19.2
numexpr 2.6.2
pysam 0.8.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.3.0
h5py 2.6.0
tables 3.3.0


In [2]:
output_dir = '/nfs/team112_internal/rp7/data/methods-dev/analysis/20170524_SIMS_sprint_1'
vw_vrpipe_fn = "%s/vw_vrpipe.txt" % output_dir
vw_vrpipe_nofilter_fn = "%s/vw_vrpipe_nofilter.txt" % output_dir

pf_60_fn = '/lustre/scratch118/malaria/team112/personal/rp7/data/wtsi-team112/Pf-6.0-public-release/20170523_update_partner_metadata/Pf_60_samples_panoptes_20170523.txt'


In [3]:
!mkdir -p {output_dir}

In [4]:
# This is the query I used in Pv 3.0, but the libs table is no longer updated, so using this reduces the number of
# rows returned from 34189 to 24682
sql_query = "\
select alfresco, oxford_code, l.lib_name, name, oxford_src_code, type \
from vw_vrpipe, libs l, sample \
where seqscape_library_id=ss_asset_id and \
sample_id=sample.id\
"

In [5]:
# This overcomes the problem with empty records in libs, if libs are needed (which here they are not)
sql_query = "\
select oxford_code, study_group, alfresco, l.lib_name, name, oxford_src_code, type \
from vw_vrpipe JOIN sample ON (sample_id=sample.id) LEFT JOIN libs l ON (seqscape_library_id=ss_asset_id)"

In [6]:
# This was a sanity check that all records have a matching row in the sample table (they do)
sql_query = "\
select alfresco, oxford_code, name, oxford_src_code, type \
from vw_vrpipe, sample \
where sample_id=sample.id\
"

In [31]:
# This is all the data that is actually needed
sql_query_vw_vrpipe = "select oxford_code, study_group, alfresco, name from vw_vrpipe"
sql_query_vw_vrpipe_nofilter = "select oxford_code, study_group, alfresco, name from vw_vrpipe_nofilter"

In [32]:
!mysql -u solaris_ro -hmalloc-db -P3309 solaris -e "{sql_query_vw_vrpipe}" > {vw_vrpipe_fn}
!mysql -u solaris_ro -hmalloc-db -P3309 solaris -e "{sql_query_vw_vrpipe_nofilter}" > {vw_vrpipe_nofilter_fn}

In [33]:
!cat "{sql_query_vw_vrpipe}"

cat: select oxford_code, study_group, alfresco, name from vw_vrpipe: No such file or directory


In [9]:
!wc -l {vw_vrpipe_fn}

34189 /nfs/team112_internal/rp7/data/methods-dev/analysis/20170524_SIMS_sprint_1/vw_vrpipe.txt


In [12]:
!wc -l {vw_vrpipe_nofilter_fn}

42146 /nfs/team112_internal/rp7/data/methods-dev/analysis/20170524_SIMS_sprint_1/vw_vrpipe_nofilter.txt


In [13]:
df_vw_vrpipe = pd.read_csv(vw_vrpipe_fn, sep='\t')
print(len(df_vw_vrpipe))
df_vw_vrpipe[0:5]

34188


,oxford_code,study_group,alfresco,name
0,PH0042-C,PF,1011-PF-KH-SU,3288_6_nonhuman.bam
1,PD0123-C,PF,1010-PF-TH-ANDERSON,5380_2_nonhuman.bam
2,PD0128-C,PF,1010-PF-TH-ANDERSON,5617_5_nonhuman.bam
3,PD0004-03,PF,1016-PF-TH-NOSTEN,1748_3_nonhuman.bam
4,PM0014-C,PF,1001-PF-ML-DJIMDE,1112_8_nonhuman.bam


In [14]:
df_vw_vrpipe_nofilter = pd.read_csv(vw_vrpipe_nofilter_fn, sep='\t')
print(len(df_vw_vrpipe_nofilter))
df_vw_vrpipe_nofilter[0:5]

42145


,oxford_code,study_group,alfresco,name
0,PH0042-C,PF,1011-PF-KH-SU,3288_6_nonhuman.bam
1,PD0123-C,PF,1010-PF-TH-ANDERSON,5380_2_nonhuman.bam
2,PD0128-C,PF,1010-PF-TH-ANDERSON,5617_5_nonhuman.bam
3,PD0004-03,PF,1016-PF-TH-NOSTEN,1748_3_nonhuman.bam
4,PJ0001-C,PV,1161-PV-ID-PRICE,2729_6_nonhuman.bam


In [30]:
print(len(df_vw_vrpipe.drop_duplicates('oxford_code')))


15599


In [27]:
pd.set_option('display.max_rows', 500)
df_vw_vrpipe.fillna('?').groupby(['study_group', 'alfresco'])['study_group'].count()

study_group  alfresco                       
@@           1087-AN-HAPMAP-DONNELLY             124
             1098-PF-ET-GOLASSA                    2
AA           1003-AG-KEGM-BURT                    11
             1030-AG-GB-DONNELLY                 372
             1055-AG-SSA-DONNELLY                 24
             1087-AN-HAPMAP-DONNELLY             901
AC           1087-AN-HAPMAP-DONNELLY             511
AE           1087-AN-HAPMAP-DONNELLY               2
AF           1087-AN-HAPMAP-DONNELLY              29
AG           1002-AG-BFUG-LAWNICZAK              177
             1003-AG-KEGM-BURT                    22
             1028-AG-WAGB-VERNICK                 57
             1029-AG-GB-LEVASHINA                  9
             1030-AG-GB-DONNELLY                1299
             1055-AG-SSA-DONNELLY                120
             1087-AN-HAPMAP-DONNELLY            7708
             1105-AG-WA-DJOGBENOU                 48
             1109-AG-COLONIES-DONNELLY            21
 

In [28]:
df_vw_vrpipe.fillna('?').groupby(['study_group'])['study_group'].count()

study_group
@@      126
AA     1308
AC      511
AE        2
AF       29
AG    11111
AM      159
HS    10084
PB       15
PF    10041
PM        3
PV      799
Name: study_group, dtype: int64

In [16]:
df_study_group_summary = pd.DataFrame(
    {
        'lanelets_nofilter': df_vw_vrpipe_nofilter.fillna('?').groupby(['study_group'])['study_group'].count(),
        'oxford_codes_nofilter': df_vw_vrpipe_nofilter.fillna('?').drop_duplicates('oxford_code').groupby(['study_group'])['study_group'].count(),
        'lanelets_filter': df_vw_vrpipe.fillna('?').groupby(['study_group'])['study_group'].count(),
        'oxford_codes_filter': df_vw_vrpipe.fillna('?').drop_duplicates('oxford_code').groupby(['study_group'])['study_group'].count()
    }
)
df_study_group_summary

,lanelets_filter,lanelets_nofilter,oxford_codes_filter,oxford_codes_nofilter
study_group,,,,
@@,126,126,46,46
AA,1308,1404,460,460
AC,511,511,237,237
AE,2,2,2,2
AF,29,29,11,11
AG,11111,14974,3789,3803
AM,159,159,57,57
HS,10084,10127,963,963
PB,15,18,15,15


In [20]:
df_study_group_summary = pd.DataFrame(
    {
        'lanelets': df_vw_vrpipe.fillna('?').groupby(['study_group'])['study_group'].count(),
        'oxford_codes': df_vw_vrpipe.fillna('?').drop_duplicates('oxford_code').groupby(['study_group'])['study_group'].count()
    }
)
df_study_group_summary

,lanelets,oxford_codes
study_group,,
@@,126,46
AA,1308,460
AC,511,237
AE,2,2
AF,29,11
AG,11111,3789
AM,159,57
HS,10084,963
PB,15,15


In [26]:
df_study_group_summary.to_excel("%s/study_group_summary.xlsx" % output_dir)

In [40]:
df_pf_60 = pd.read_csv(pf_60_fn, sep='\t', index_col=0)
print(len(df_pf_60))
df_pf_60[0:5]

7182


,OxfordSrcCode,OxfordDonorCode,Individual_ID,AlfrescoStudyCode,Type,InV5.1,InV5,SubContinentCode,SubContinentName,RegionCode,...,mean_indel_len,total_indel_len,dont_release_Year,release_exclusion_reason,platform,pe_se,path,max_length,min_length,most_common_length
Sample,,,,,,,,,,,,,,,,,,,,,
FP0008-C,KB02.14,KB02.14,KB02.14,1147-PF-MR-CONWAY,DNA,False,False,WAF,West Africa,MR,...,-0.188319,-586569,2014,None,HiSeq,pe,/lustre/scratch116/malaria/pfalciparum/output/...,100,30,100
FP0009-C,KB03.14,KB03.14,KB03.14,1147-PF-MR-CONWAY,DNA,False,False,WAF,West Africa,MR,...,-0.253752,-790377,2014,None,HiSeq,pe,/lustre/scratch116/malaria/pfalciparum/output/...,100,30,100
FP0015-C,KB18.14,KB18.14,KB18.14,1147-PF-MR-CONWAY,DNA,False,False,WAF,West Africa,MR,...,-0.241471,-752124,2014,None,HiSeq,pe,/lustre/scratch116/malaria/pfalciparum/output/...,100,30,100
FP0016-C,KB19.14,KB19.14,KB19.14,1147-PF-MR-CONWAY,DNA,False,False,WAF,West Africa,MR,...,-0.096027,-299102,2014,None,HiSeq,pe,/lustre/scratch116/malaria/pfalciparum/output/...,100,33,100
FP0017-C,KB20.14,KB20.14,KB20.14,1147-PF-MR-CONWAY,DNA,False,False,WAF,West Africa,MR,...,-0.262139,-816501,2014,None,HiSeq,pe,/lustre/scratch116/malaria/pfalciparum/output/...,100,30,100


In [44]:
df_pf_60['AlfrescoStudyCode'].value_counts()

1052-PF-TRAC-WHITE                 1172
1125-PF-TH-NOSTEN                   674
1044-PF-KH-FAIRHURST                602
1017-PF-GH-AMENGA-ETEGO             390
1022-PF-MW-OCHOLLA                  351
1095-PF-TZ-ISHENGOMA                300
1094-PF-GH-AMENGA-ETEGO             256
1093-PF-CM-APINJOH                  239
1008-PF-SEA-RINGWALD                234
1026-PF-GN-CONWAY                   197
1031-PF-SEA-PLOWE                   177
1101-PF-CD-ONYAMBOKO                174
1127-PF-ML-SOULEYMANE               164
1097-PF-ML-MAIGA                    137
1151-PF-GH-AMENGA-ETEGO             111
1010-PF-TH-ANDERSON                 108
1131-PF-BJ-BERTIN                   102
1083-PF-GH-CONWAY                   101
1096-PF-GH-GHANSAH                  101
1136-PF-GM-NGWA                     100
1001-PF-ML-DJIMDE                    96
1155-PF-ID-PRICE                     92
1135-PF-SN-CONWAY                    86
1147-PF-MR-CONWAY                    86
1062-PF-PG-BARRY                     82


In [42]:
df_pf_60.ix[df_pf_60['AlfrescoStudyCode'] == '1195-PF-TRAC2-DONDORP']

,OxfordSrcCode,OxfordDonorCode,Individual_ID,AlfrescoStudyCode,Type,InV5.1,InV5,SubContinentCode,SubContinentName,RegionCode,...,mean_indel_len,total_indel_len,dont_release_Year,release_exclusion_reason,platform,pe_se,path,max_length,min_length,most_common_length
Sample,,,,,,,,,,,,,,,,,,,,,
